### 자연어 감성 분석
- 감성사전 기반 : 미리 저으이된 감성단어 사전 사용(규칙 기반)
    - TextBlob, AFINNm VADER
- 머신러닝 기반 : 데이터로 부터 패턴 학습(통계 기반)
    - TF-IDF 벡터화
    - 빈도수 벡터화
    - Multinomial Naive Bayes
    - 로지스틱 회귀

### 사용 데이터
- NLTK 영화 리뷰 (2000개)
- 다음영화리뷰

#### 알고리즘
- textblob
- afinn
- vader

- TF-IDF
- CounterVectorizer
- Multinomial Naive Bayes
- 로지스틱 회귀 ^

In [ ]:
# TextBlob
# 특정단어를 선정하고, 그 단어가 나올경우 점수를 추가하거나 감점한다.
# Polarity  극성도
# 긍정(1)과 부정(-1)
# 0은 중립
# Subjectivity 주관성
# 객관(-1)과 주관(1)
# 0은 중립

# 문맥은 무시하고 단어 여부로만 판단한다.
# 이 영화는 나쁘지 않다. -> 나쁘다, 않다 에서 감점(-)으로 인식
# 장점 : 빠른속도 학습 불필요
# 사용 : 실시간 감성 분석, 스트리밍 데이터 -> 빠른 판단이 필요한 상황에 사용하기도 한다.

In [ ]:
# %pip install TextBlob

In [ ]:
from textblob import TextBlob, Word
text = "TextBlob is amazingly simple to use. What a wonderful library for NLP!"
blob = TextBlob(text)
print(blob.sentences)
print(blob.words)
print(blob.tags)

import nltk
# nltk.download('brown')
print(blob.noun_phrases)

# 문장, 단어, 품사 단위의 기본 분석 기능을 지원함을 확인할 수 있다.
# 단어 분리의 경우에는 무엇을 기준으로? ^ 공백, 문장부호, 형태소 분석 규칙을 기준

In [ ]:
# 감성 분석
blob.sentiment
# Sentiment(polarity=0.5, subjectivity=0.6785714285714286)

# Polarity  극성도
# 긍정(1)과 부정(-1)
# 0은 중립
# Subjectivity 주관성
# 객관(-1)과 주관(1)
# 0은 중립

# 약간 긍정이고, 주관적인 표현이다.

print()
print()

In [ ]:
# AFINN(Lexicon-Based)
# 각 단어의 -5 ~ +5의 점수를 부여하고 합산
# 이 영화는 좋지만 좋지않은 부분도 있다.
    # 좋다 +3 좋다 +3 않다 -3 = +3 > 0 -> 긍정 으로 평가
# 조회 방법 : score = sum(word_sentiment_value)
# 분류규칙
    # 0 > score 긍정
    # score < 0 부정
# 이모티콘지원
# 강도표현 인식 very, really 등

# 강조 수정자(intensifiers)
    # 매우좋다 = 1.5 x (좋다의 점수)

# AFINN vs TextBlob
# AFINN : 더 정확한 점수 매핑
# TextBlob : 더 일반적인 접근 ^

In [8]:
# %pip install afinn

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for afinn: filename=afinn-0.1-py3-none-any.whl size=53479 sha256=d5e78352c9a512aa4d0d73e7468f036fd2534ea239a3df9b2abd21038a65bdfb
  Stored in directory: c:\users\playdata2\appdata\local\pip\cache\wheels\f3\20\88\bd79bf1dfa529d0e4e301372371edf7639514a9f0d337769c3
Successfully built afinn
Note: you may need to restart the kernel to use updated packages.


  DEPRECATION: Building 'afinn' using the legacy setup.py bdist_wheel mechanism, which will be removed in a future version. pip 25.3 will enforce this behaviour change. A possible replacement is to use the standardized build interface by setting the `--use-pep517` option, (possibly combined with `--no-build-isolation`), or adding a `pyproject.toml` file to the source tree of 'afinn'. Discussion can be found at https://github.com/pypa/pip/issues/6334


In [2]:
from afinn import Afinn
af = Afinn()
text1 = "TextBlob is amazingly simple to use."
text2 = "What a wonderful library for NLP!"
score1 = af.score(text1)
score2 = af.score(text2)
score1, score2

(0.0, 4.0)

In [ ]:
# VADER 소셜미디어 텍스트에 최적화
# 이 영화는 정말정말 훌륭해!!!
# 훌륭하다 (기본) + 0.7 정말정말 (강조) x 1.5
# !!! (문장부로강조)  x 1.2

# 4개의 감정 지수
    # positive 긍정 확률 0 ~ 1
    # negative 부정 확률
    # neutral 중립 확률
    # compund 종합점수 -1 ~ 1
        # +1 에 가까울수록 매우 긍정적
        # -1 에 가까울수록 매우 부정적
        # 0 근처라면 중립적

# 종합점수 조회 방법 :
# score = compound_score / sqrt(compound_score**2 + 0.0625)
# score >= 0.05 긍정
# score <= -0.05 부정
# 그 사이( -0.05 < score < 0.05 )는 중립

# 대소문자 구분 AMAZING amazing 다른 점수
# :) 긍정 :-( 부정

In [3]:
import nltk
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\Playdata2\AppData\Roaming\nltk_data...


True

In [ ]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()

sentences = [
    "I love this product! It's absolutely amazing 😍",
    "This is the worst movie I've ever seen...",
    "The food was okay, not great but not bad either.",
    "I’m REALLY happy with the results!!!",
    "Not good at all. I’m disappointed.",
]

for s in sentences:
    scores = analyzer.polarity_scores(s)
    print(f'문장 : {s}')
    print(f'점수 : {scores}')

문장 : I love this product! It's absolutely amazing 😍
점수 : {'neg': 0.0, 'neu': 0.318, 'pos': 0.682, 'compound': 0.862}
문장 : This is the worst movie I've ever seen...
점수 : {'neg': 0.369, 'neu': 0.631, 'pos': 0.0, 'compound': -0.6249}
문장 : The food was okay, not great but not bad either.
점수 : {'neg': 0.149, 'neu': 0.487, 'pos': 0.364, 'compound': 0.4728}
문장 : I’m REALLY happy with the results!!!
점수 : {'neg': 0.0, 'neu': 0.472, 'pos': 0.528, 'compound': 0.7651}
문장 : Not good at all. I’m disappointed.
점수 : {'neg': 0.579, 'neu': 0.421, 'pos': 0.0, 'compound': -0.6711}
